In [1]:
from __future__ import print_function

import os
import sys
import time
from datetime import timedelta

import numpy as np
import tensorflow as tf
from sklearn import metrics

from cnn_model import TCNNConfig, TextCNN
from data.cnews_loader_test import read_vocab, read_category, batch_iter, process_file, build_vocab

/Users/sunchao/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/Users/sunchao/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
base_dir = 'data/cnews'
train_dir = os.path.join(base_dir, 'cnews.train.txt')
test_dir = os.path.join(base_dir, 'cnews.test.txt')
val_dir = os.path.join(base_dir, 'cnews.val.txt')
vocab_dir = os.path.join(base_dir, 'cnews.vocab.txt')


save_dir = 'checkpoints/textcnn'
save_path = os.path.join(save_dir, 'best_validation')

In [5]:
def get_time_dif(start_time):
    """获取已经运行的时间"""
    end_time = time.time()
    time_dif = end_time - start_time
    return timedelta(seconds=int(round(time_dif)))


In [6]:
def feed_data(x_batch, y_batch, keep_prob):
    feed_dict = {
        model.input_x : x_batch,
        model.input_y : y_batch,
        model.keep_prob : keep_prob
    }
    return feed_dict

In [7]:
# 更多是在测试在验证集上的结果
def evaluate(sess, x_, y_):
    """评估在某一数据上的准确率和损失"""
    data_len = len(x_)
    batch_eval = batch_iter(x_, y_, 128)
    total_loss = 0.0
    total_acc = 0.0

    for x_batch, y_batch in batch_eval:
        batch_len = len(x_batch)
        feed_dict = feed_data(x_batch, y_batch, 1.0)
        loss, acc = sess.run([model.loss, model.acc], feed_dict=feed_dict)
        total_loss += loss * batch_len
        total_acc += acc * batch_len

    return total_loss/data_len, total_acc/data_len

In [10]:
config = TCNNConfig()
categories, cat_to_id = read_category()
words, word_to_id = read_vocab(vocab_dir)
config.vocab_size = len(words)
model = TextCNN(config=config)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [16]:
categories, cat_to_id

(['体育', '财经', '房产', '家居', '教育', '科技', '时尚', '时政', '游戏', '娱乐'],
 {'体育': 0,
  '娱乐': 9,
  '家居': 3,
  '房产': 2,
  '教育': 4,
  '时尚': 6,
  '时政': 7,
  '游戏': 8,
  '科技': 5,
  '财经': 1})

In [19]:
config.vocab_size

5000

In [20]:
x_test, y_test = process_file(test_dir, word_to_id, cat_to_id, config.seq_length)

In [28]:
y_test[5516]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0.])